In [ ]:
#essentially, tool for making datasets from wikipedia. Ask for data around a particular topic and a for a brief analysis of it. Have the agent query wikipedia, format the results into a dataframe, and return a dataframe as well as some visualizataions

#search wikipedia for lists with tables
#parse tables into dataset
#display dataset and analyze


In [1]:
!nvidia-smi

Wed Sep  3 09:49:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8               6W / 170W |     11MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
[sudo] password for thoth: 


In [2]:
!pip install -U wikipedia langgraph-checkpoint-sqlite llama-cpp-python ollama llama-stack langchain_experimental

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 3.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [3]:
!pip install -U langchain-nomic langchain_community langchainhub chromadb langchain langgraph tavily-python nomic[local] langchain-text-splitters langchain-ollama #tiktoken

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached langchain_core-0.3.69-py3-none-any.whl.metadata (5.8 kB)
  Using cached sqlalchemy-2.0.41-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached langsmith-0.4.6-py3-none-any.whl.metadata (15 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached pydantic-2.11.7-py3-no

In [3]:
!ollama list

NAME                    ID              SIZE      MODIFIED      
qwen2.5-coder:latest    dae161e27b0e    4.7 GB    3 seconds ago    


In [2]:
#!ollama pull qwen2.5-coder #not calling tools properly for some reason
!ollama pull llama3.1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 60e05f210007: 100% ▕██████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling 1e65450c3067: 100% ▕██████████████████▏ 1.6 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling d9bb33f27869: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama list

NAME                    ID              SIZE      MODIFIED       
qwen2.5-coder:latest    dae161e27b0e    4.7 GB    24 minutes ago    


In [6]:
#Importing the required packages
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# from langchain_community.tools.openai_dalle_image_generation import (
#    OpenAIDALLEImageGenerationTool
# )
#from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
import os
import pandas as pd
from langchain_core.tools import tool
#from langchain.tools.render import format_tool_to_openai_function
from langchain.schema import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_ollama import ChatOllama

from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


@tool
def get_wikipedia_docs(query:str) -> List:
  """
  Tool that takes in a query and returns a list of
  the top 3 wikipedia documents most relevant to that query
  """

  wikipedia_docs = WikipediaLoader(query, load_max_docs=3)
  return wikipedia_docs.load()

@tool
def get_wiki_tables(wikipedia_docs) -> list:
  """Tool that takes in a list of wikipedia docs, parses out the url
  from each doc, and extracts any tables from that url.
  Returns a list of pandas dataframes containing the tables.
  """
  links = [x.metadata['source'] for x in wikipedia_docs]
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
  tables_pds = []
  for url in links:
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        tables_pds += pd.read_html(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
    except ValueError as e:
        print(f"Error parsing HTML: {e}")
    return tables_pds

@tool
def aggregate_dataframes():

  return

@tool
def choose_visualization(df) -> dict:     #adapted from https://blog.langchain.com/data-viz-agent/, particularly the prompt

    """Choose an appropriate visualization for the data."""

    dataframe_question_string = """
        My dataframe has the shape {shape}. Here is what the first 3 rows look like:\n
        {df_head}
        """.format(shape=df.shape, df_head=str(df.head(3)))


    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
You are an AI assistant that recommends appropriate data visualizations. A user has a dataframe they want to visualize. Based on the structure of the dataframe, suggest the most suitable type of graph or chart to visualize the data that produces an interesting insight. If no visualization is appropriate, indicate that. Also suggest the format the dataframe should be aggregated to produce this visualization.

Available chart types and their use cases:
- Bar Graphs: Best for comparing categorical data or showing changes over time when categories are discrete and the number of categories is more than 2. Use for questions like "What are the sales figures for each product?" or "How does the population of cities compare? or "What percentage of each city is male?"
- Horizontal Bar Graphs: Best for comparing categorical data or showing changes over time when the number of categories is small or the disparity between categories is large. Use for questions like "Show the revenue of A and B?" or "How does the population of 2 cities compare?" or "How many men and women got promoted?" or "What percentage of men and what percentage of women got promoted?" when the disparity between categories is large.
- Scatter Plots: Useful for identifying relationships or correlations between two numerical variables or plotting distributions of data. Best used when both x axis and y axis are continuous. Use for questions like "Plot a distribution of the fares (where the x axis is the fare and the y axis is the count of people who paid that fare)" or "Is there a relationship between advertising spend and sales?" or "How do height and weight correlate in the dataset? Do not use it for questions that do not have a continuous x axis."
- Pie Charts: Ideal for showing proportions or percentages within a whole. Use for questions like "What is the market share distribution among different companies?" or "What percentage of the total revenue comes from each product?"
- Line Graphs: Best for showing trends and distributionsover time. Best used when both x axis and y axis are continuous. Used for questions like "How have website visits changed over the year?" or "What is the trend in temperature over the past decade?". Do not use it for questions that do not have a continuous x axis or a time based x axis.

Consider these types of questions when recommending a visualization:
1. Aggregations and Summarizations (e.g., "What is the average revenue by month?" - Line Graph)
2. Comparisons (e.g., "Compare the sales figures of Product A and Product B over the last year." - Line or Column Graph)
3. Plotting Distributions (e.g., "Plot a distribution of the age of users" - Scatter Plot)
4. Trends Over Time (e.g., "What is the trend in the number of active users over the past year?" - Line Graph)
5. Proportions (e.g., "What is the market share of the products?" - Pie Chart)
6. Correlations (e.g., "Is there a correlation between marketing spend and revenue?" - Scatter Plot)

Provide your response in the following format:
Recommended Visualization: [Chart type or "None"]. ONLY use the following names: bar, horizontal_bar, line, pie, scatter, none
Reason: [Brief explanation for your recommendation]
'''),
        ("human", '''
{question}

Recommend a visualization:'''),
    ])

    formatted_prompt = prompt.format_prompt(question=dataframe_question_string)

    response = llm.invoke(formatted_prompt)

    response_dict = json.loads(response.content)

    return {"visualization": response_dict['Recommended Visualization'], "visualization_reason": response_dict['Reason']}
    return {"visualization": visualization, "visualization_reason": reason}


def plot_visualization(dt, visualization_type):

  return




tools = [get_wikipedia_docs, get_wiki_tables, aggregate_dataframes, choose_visualization, plot_visualization]

In [ ]:
llm = ChatOllama(model="llama3.1:8b", format="json", temperature=0)
memory = MemorySaver()

agent_executor = create_react_agent(llm, tools)#, checkpointer=memory)
agent_executor

In [ ]:
message_content = "Can you tell me what the most densely populated cities in the world are?"

config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=message_content)]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you tell me what the most densely populated cities in the world are?
================================== Ai Message ==================================

{"name": "wikipedia", "arguments": {"query": "most densely populated cities in the world"}}


In [ ]:

#if the topic asks for data, use the wikipedia tool
#if the topic asks for trends, use the chart tool